# Likelihood-Ratio tests in Python

*This code was written by Jonathan Newman (jonathannewman55@gmail.com)*

### Set up Ipython environment

Jupyter notebook runs in an iPython environment meaning you will need iPython to autoreload imported files (especially custom packages)

In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load appropriate modules and packages

In [22]:
import numpy as np
import pandas as pd
import math
import statsmodels.formula.api as smf
import scipy
import sys ## needed for installation!

### Mock dataframe

Let's suppose we're interested in the relationship between labour productivity and ICT adoption. We have cross-sectional df on labour productivity and gross ICT investment at the business level (n=2000) across 10 sectors and 5 size sub-categories. I'll first create a dataframe with randomised entries for industry-level productivity, ICT investment, size and sectors.

In [30]:
## build exemple dataframe in four variables: 
## productivity, investment, size and sector
obs = 2000
## first get array of productivity observations
mean, std = 45000, 3000
product_array = np.random.normal(mean, std, obs)

## then get array of investment observations
mean, std = 3000, 500
invest_array = np.random.normal(mean, std, obs)

size_array = np.random.randint(1,5,obs) ## let's suppose there are five size sub-categories
sector_array = np.random.randint(1,10,obs) ## let's create 10 sector sub-categories

In [31]:
columns = ["Labour Productivity",'ICT Investment',
           'Size','Sector']
var_list = [product_array,invest_array,
            size_array,sector_array]
data = pd.DataFrame(var_list,columns).transpose()

## remove weakly negative responses before log-transform (small chance there are any)
data = data.mask(data <= 0,pd.NA)
data.dropna()

## Intuitive to think of % change in productivity to % change in investment
data['ln_ict'] = data['ICT Investment'].apply(lambda row: math.log(row))
data['ln_productivity'] = data['Labour Productivity'].apply(lambda row: math.log(row))

## ensure size and sector are recorded as categorical variables
data['Size'] = data['Size'].apply(lambda row: str(row))
data['Sector'] = data['Sector'].apply(lambda row: str(row))

#### Likelihood-Ratio test for sector dummies
Let's suppose we also want to know whether, in a model with ICT investment, business sector is a significant predictor of labour productivity. One method to do this is to test the joint significance of the categorical sector variables using a Likelihood-Ratio test.

Consider two estimators $\theta_{0}$ (restricted model) and $\theta_{1}$ (unrestricted model) for a parameter vector $\theta$. Let $\mathcal{L}(\cdot)$ denote the likelihood function and $k$ the number of extra parameters in the unrestricted model. The Likelihood Ratio test is then $LR = -2\ln{\frac{\mathcal{L}(\theta_{0})}{\mathcal{L}(\theta_{1})}}$, asymptotically distributed $\chi^{2}_{k}$. However it's often convenient to work with log-transformed likelihood functions, yielding the equivalent:

- $LR = 2[\ln\mathcal{L}(\theta_{1})-\ln\mathcal{L}(\theta_{0})]$

In [34]:
## First run restricted model without sector dummies
formula = "ln_productivity ~ ln_ict + Size"
mod_ols = smf.ols(formula, data).fit()
## get log-likelihood function of OLS model
null_ll = mod_ols.llf

## Then run unrestricted model with sector dummies
formula = formula.replace('ln_ict',
                          'Sector + ln_ict')
mod_ols = smf.ols(formula, data).fit()


In [36]:
## get log-likelihood function of WLS model
alt_ll = mod_ols.llf

## compute log-likelihood ratio statistic and p-value
l_stat = 2*(alt_ll - null_ll)
p = 1 - scipy.stats.chi2.cdf(l_stat, len(set(df['Sector'])))

if p < 0.05:
    print(f"p = {p:.3} < 0.05 hence the sector dummies are jointly significant")
else:
    print(f"p = {p:.3} > 0.05 hence the sector dummies are not jointly significant")

p = 0.825 > 0.05 hence the sector dummies are not jointly significant


In linear models with normally distributed errors, the F test is equivalent to the Likelihood Ratio test. With $q$ restrictions in the restricted model, $n$ observations and $k$ parameters in the unrestricted model, the F-test is $F = \frac{(R^{2}_{UR}-R^{2}_{R})/q}{(1-R^{2}_{UR})/n-k-1}$. Unlike the Likelihood Ratio test outlined previously, the F test follows a $F_{q,n-k-1}$ distribution. Note that in *finite* samples the equivalence is not exact!

In [37]:
## First run restricted model without sector dummies
formula = "ln_productivity ~ ln_ict + Size"
rest = smf.ols(formula, data).fit()

## Next run unrestricted model with sector dummies
formula = formula.replace('ln_ict',
                          'Sector + ln_ict')
unrest = smf.ols(formula, data).fit()

In [40]:
## Get F-stat numerator
fstat_num = (unrest.rsquared - rest.rsquared)/len(set(data['Sector']))

## Get degrees of freedom
dof = len(data) - unrest.params.size - 1

## Get F-stat denominator
fstat_den = (1-unrest.rsquared)/dof

## Get test statistic and p-value
fstat = fstat_num/fstat_den
p = 1 - scipy.stats.f.cdf(fstat, len(set(data['Sector'])), dof)

if p < 0.05:
    print(f"p = {p:.3} < 0.05 hence the sector dummies are jointly significant")
else:
    print(f"p = {p:.3} > 0.05 hence the sector dummies are not jointly significant")

p = 0.827 > 0.05 hence the sector dummies are not jointly significant
